# Менеджеры контекстов. Переопределение оператора точка. Модификаторы доступа

Алексей Умнов https://www.youtube.com/watch?v=0K8XzOqBMus  
Слайды доступны по адресу: http://parallels.nsu.ru/~fat/Python/

## Контекст с остановкой исключений

In [1]:
class Context():

    def __init__(self, stop_exception):
        print('__init__({})'.format(stop_exception))
        self.stop_exception = stop_exception

    def __enter__(self):
        print('__enter__()')
        return 'Some data'

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('__exit__({}, {}, {})'.format(exc_type, exc_val, exc_tb))
        return self.stop_exception

Например, в коде неожиданно возбуждается исключение... и оно отлавливается менеджером контекста в методе `__exit__`.  
Так как метод `__exit__` в нашем случае возвращает `True`, то код `with ... as ...` завершится успешно и код за пределами `with` продолжит выполняться:

In [2]:
with Context(True) as x:
    print('Inside context, x =', x)
    raise RuntimeError('Smth is wrong')
    print('After exception') # не выведется

print ('After context manager')

__init__(True)
__enter__()
Inside context, x = Some data
__exit__(<class 'RuntimeError'>, Smth is wrong, <traceback object at 0x7f376e7291c8>)
After context manager


Но если, метод `__exit__` вернет `False`, то код не пойдет дальше выброшенного исключения:

In [3]:
with Context(False) as x:
   print('Inside context, x =', x)
   # Если в коде возбуждается исключение, то оно отлавливается менеджером контекста в методе __exit__
   raise RuntimeError('Smth is wrong')

print ('After context manager') # Не выведется

__init__(False)
__enter__()
Inside context, x = Some data
__exit__(<class 'RuntimeError'>, Smth is wrong, <traceback object at 0x7f376e729f88>)


RuntimeError: Smth is wrong

## Контекст без исключений

In [4]:
class Context():

    def __enter__(self):
        print('__enter__()')
        return 'Some data'

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('__exit__({}, {}, {})'.format(exc_type, exc_val, exc_tb))
        # Успешность отработки кода в контексте проверяется переменной exc_type:
        # if exc_type != None:


with Context() as x:
    print('Inside context, x =', x)
    # Если в коде исключение не возникает, то в __exit__ аргументы передаются с значением None

__enter__()
Inside context, x = Some data
__exit__(None, None, None)


## Менеджер контекста на примере временного файла

* Существует только внутри контекста
* Можно читать и писать

````python
with TmpFile('/tmp/') as path:
    with open(path, 'w') as fw:
        fw.write('Some stuff')
        ...
    ...
    with open(path) as fr:
        ... = fr.read()
````

(для использования временных файлов существует модуль tempfile)

In [5]:
import os

class TmpFile():

    def __init__(self, folder):
        self.folder = folder

    def generate_name(self):
        return 'test_python.tmp'

    def __enter__(self):
        name = self.generate_name()
        self.path = os.path.join(self.folder, name)
        print('__enter__', 'path =', self.path)
        # Создаем пустой файл, на тот случай, если в менеджере контекстов файл открываться не будет,
        # то при выходе из контекста, метод __exit__ выпадет в исключение при удалении несуществующего
        # файла (os.remove ...)
        with open(self.path, 'w'):
            pass # create empty file
        # Эта переменная возвращается в переменную после ключевого слова as в менеджере контекста
        return self.path

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('__exit__', 'Удаляем файл:', self.path)
        os.remove(self.path)

Использование:

In [6]:
# <- Инициализация экземпляра класса TmpFile + вызов входа в контекст (метод __enter__)
with TmpFile('/tmp/') as path:
    with open(path, 'w') as fw:
        print('Write data: Some stuff')
        fw.write('Some stuff')
    #
    # Здесь файл на запись уже закрыт, так как контекст закрыт
    #
    with open(path) as fr:
        data = fr.read()
        print('Read data:', data)

__enter__ path = /tmp/test_python.tmp
Write data: Some stuff
Read data: Some stuff
__exit__ Удаляем файл: /tmp/test_python.tmp


## Операторы доступа к атрибутам (оператор точка)

Операторы доступа к атрибутам:

````python
a.x       # Чтение атрибута  
a.x = ... # Запись атрибута
````

Примеры использования:
* Не нужно всегда писать функции чтения и записи для всех атрибутов
  (если атрибут нужно будет превратить в метод, он все равно сможет "притворяться" атрибутом)
* Псевдо-атрибуты
  (для которых данные не хранятся, а вычисляются)

## Оператор __getattr__(self, name)

Оператор `__getattr__(self, name)`:

* Срабатывает при чтении атрибута, например: `obj.attr`
* Не вызывается, если атрибут существует
* Получает на вход имя атрибута
* Должен вернуть его значение, например сгенерировать или создать `AttributeError`
* Может использовать внутри другие атрибуты
* Методы - тоже атрибуты

In [7]:
class A():

    def __init__(self):
        self.x = 1

    def __getattr__(self, name):
        print('Доступ к несуществующему атрибуту:', name)
        return self.x + 1


a = A()

print('a.x =', a.x)

a.x = 1


In [8]:
print('a.attr =', a.attr)

Доступ к несуществующему атрибуту: attr
a.attr = 2


In [9]:
print('a.method =', a.method)

Доступ к несуществующему атрибуту: method
a.method = 2


## Эмуляция отсутствия атрибута

In [10]:
class A():

    def __init__(self):
        self.base = 1

    def __getattr__(self, name):
        if name == 'doubled':
            return self.base * 2 # Атрибут doubled вычисляется через реальный атрибут
        else:
            raise AttributeError('Attribute not found')


a = A()

Реальный атрибут:

In [11]:
print('a.base =', a.base)

a.base = 1


Вычислененный атрибут:

In [12]:
print('a.doubled =', a.doubled)

a.doubled = 2


Вызовет исключение `AttributeError: Attribute not found`:

In [13]:
print('a.other =', a.other)

AttributeError: Attribute not found

Ошибки нет. Создается реальный атрибут со значением 1:

In [14]:
a.doubled = 1

Обращение идет к реальному атрибуту, а не к методу `__getattr__`:

In [15]:
print('a.doubled =', a.doubled)

a.doubled = 1


In [16]:
print('Реальные атрибуты объекта:', a.__dict__)

Реальные атрибуты объекта: {'base': 1, 'doubled': 1}


## Оператор __setattr__(self, name, value)

Оператор `__setattr__(self, name, value)`

* Вызывается при изменении атрибута (a.x = ...)
* Если определен, то вызывается всегда
* Получает на вход имя атрибута и значение
* Может изменять другие атрибуты через `super(C, self).__setattr__()`

In [17]:
class A():

    def __init__(self):
        self.x = 1      # Вызывается метод __setattr__

    def __setattr__(self, name, value):
        # Здесь мы не можем просто обратиться к объекту через self.x чтобы установить ему какое-то значение,
        # так как вызовем бесконечную рекурсию, поэтому применим следующий метод для доступа к самому себе:
        super(A, self).__setattr__('x', value * 2) # Вызываем __setattr__ у предка.


a = A()
a.x = 100
print('a.x =', a.x)

a.x = 200


При этом, больше нет возможности создать другой атрибут, так как завязано всё на атрибут `x` в методе `__setattr__()`. Изменится значение атрибута `x`, а атрибут `y` не создастся:

In [18]:
a.y = 300

print('a.x =', a.x)

a.x = 600


Атрибут `y` не доступен:

In [19]:
print('a.y =', a.y)

AttributeError: 'A' object has no attribute 'y'

## Оператор __getattribute__(self, name)

Оператор `__getattribute__(self, name)`:

* Срабатывает при чтении атрибута (a.x)
* Если определен, то вызывается всегда
* Получает на вход имя атрибута
* Должен вернуть его значение или создать `AttributeError`
* Может получать другие атрибуты через `super(C, self).__getattribute__()`
* Исключение - методы `__*__`
* Более сложный вариант `__getattr__` который работает всегда (если определен в классе)

In [20]:
class A():

    def __init__(self):
        self.x = 1

    # Метод вызывается при отсутствии атрибута
    def __getattr__(self, name):
        print('Доступ к несуществующему атрибуту:', name)
        return self.x * 2 # Аналогично вызову: self.__getattribute__(x) * 2

    # Метод вызывается всегда
    def __getattribute__(self, name):
        print('Доступ к атрибуту:', name)
        return super(A, self).__getattribute__(name) # Обращение к самому себе


a = A()

print('a.x =', a.x)

Доступ к атрибуту: x
a.x = 1


In [21]:
print('a.y =', a.y)

Доступ к атрибуту: y
Доступ к несуществующему атрибуту: y
Доступ к атрибуту: x
a.y = 2


Еще один пример:

In [22]:
class A():

    def __init__(self):
        self.x = 1

    def __getattribute__(self, name):
        return super(A, self).__getattribute__('x') + 100


a = A()

Так как метод `__getattribute__` вызывается всегда, то можно переопределить значение для существующего и несуществующего атрибута:

In [23]:
print('a.x =', a.x)

a.x = 101


In [24]:
print('a.y =', a.y)

a.y = 101


| Оператор         | Код   | Вызывается                        |
|------------------|-------|-----------------------------------|
| __getattr__      | a.x   | Если не сработал обычный механизм |
| __getattribute__ | a.x   | Всегда                            |
| __setattr__      | a.x = | Всегда                            |

## Функции для работы с атрибутами

In [25]:
class A():

    def __init__(self):
        self.x = 1

a = A()

Эквивалентно: `a.x`

In [26]:
getattr(a, 'x')

1

Эквивалентно: `a.x = 100`

In [27]:
setattr(a, 'x', 100)

In [29]:
getattr(a, 'x')

100

Существует ли атрибут?

In [28]:
hasattr(a, 'y')

False

## Модификаторы доступа

Типы атрибутов:

| Тип        | Доступен снаружи |  Доступен из наследников |
|------------|------------------|--------------------------|
| Открытый   | Да               | Да                       |
| Защищенный | Нет              | Да                       |
| Закрытый   | Нет              | Нет                      |

Всегда есть доступ ко всем атрибутам.  
Специальные обозначения для закрытых и защищенных атрибутов.
* Нет "защиты от дурака"
* Есть защита от ошибок
* Полезно в некоторых программах (отладка, текстовые редакторы)

### Защищенные атрибуты обозначаются префиксом _ (одинарное подчеркивание)

In [30]:
class A():

    def __init__(self):
        self.normal_attr = 0
        # Защищенный атрибут (это просто соглашение, для интерпретатора ничего не меняется)
        self._protected_attr = 0

Доступ изнутри:
* По имени (так же, как для открытых атрибутов)

Доступ снаружи:
* По имени (так же, как для открытых атрибутов)
* Легко заметен в коде
* Обнаруживается редакторами и чекерами

### Закрытые атрибуты обозначаются префиксом __ (двойное подчеркивание)

In [31]:
class A():

    def __init__(self):
        self.normal_attr = 0
        self._protected_attr = 0
        self.__private_attr = 0

    def __private_method(self):
        pass

Проблема. Как получить доступ, если в классе наследнике есть такое же имя?

### Декорирование имен

In [32]:
class A():

    def public_method(self):
        pass

    def __private_method(self):
        pass

Меняется имя у атрибута `__private_method` (методы это тоже атрибуты):

In [33]:
dir(A)

['_A__private_method',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'public_method']

Закрытые атрибуты:

````python
class CLASSNAME():
    __ATTRIBUTENAME = 0
````

Доступ изнутри:
* `__ATTRIBUTENAME`

Доступ снаружи:
* `_CLASSNAME__ATTRIBUTENAME`
* Легко заметен в коде
* Обнаруживается редакторами и чекерами

### Обзор обозначений


| Обозначение  | Значение                   |
|--------------|----------------------------|
| name         | Открытый атрибут           |
| _name        | Защищенный атрибут         |
| __name       | Закрытый атрибут (внутри)  |
| _class__name | Закрытый атрибут (снаружи) |
| __name__     | Зарезервированный атрибут  |


## Оформление атрибутов

### Простое оформление атрибутов:

In [34]:
class Parrot():

    def __init__(self):
        self.voltage = 1000 # Открытый атрибут


p = Parrot()
p.voltage = 'qwerty' # Изменение атрибута снаружи

Особенности этого подхода:
* **-** Класс не узнает, что его атрибут изменили
* **-** Пользователь класса не узнает, что ошибся (программа запустится и будет корректно исполнятся)
* **+** Быстро можно писать код

### Сложное оформление атрибутов

In [36]:
class Parrot():

    def __init__(self):
        # Закрытый атрибут (пользователи и наследники класса не могут пользоваться напрямую)
        self.__voltage = 1000

    def voltage(self):
        return self.__voltage

Особенности этого подхода:
* **+** Атрибут защищен
* **-** Нельзя изменять
* **-** Доступ к атрибуту через voltage() вместо voltage

### Перегрузка оператора `.` (точка)

In [39]:
class Parrot():

    def __init__(self):
        # Закрытый атрибут (пользователи и наследники класса не могут пользоваться напрямую)
        self.__voltage = 1000

    # Метод вызывается при отсутствии атрибута
    def __getattr__(self, name):
        print('Метод __getattr__({}) - Атрибут не существует'.format(name))
        if name == 'voltage':
            return self.__voltage

        raise AttributeError('attribute "' + name + '" not found')

    # Метод всегда вызывается когда определен атрибут
    def __setattr__(self, name, value):
        print('Метод __setattr__({}, {}) - Изменение атрибута'.format(name, value))
        if name == 'voltage':
            assert isinstance(value, int)
            self.__voltage = value # Внимание! Вызовет этот же метод (__setattr__)
        else:
            super(Parrot, self).__setattr__(name, value) # Если атрибут не существует, то создаем его


p = Parrot()
print('p.voltage =', p.voltage)

Метод __setattr__(_Parrot__voltage, 1000) - Изменение атрибута
Метод __getattr__(voltage) - Атрибут не существует
p.voltage = 1000


Атрибута `voltage` не существует, изменяем значение атрибута `__voltage` через метод `__setattr__`:

In [40]:
p.voltage = 2500

Метод __setattr__(voltage, 2500) - Изменение атрибута
Метод __setattr__(_Parrot__voltage, 2500) - Изменение атрибута


In [41]:
print('p.voltage =', p.voltage)

Метод __getattr__(voltage) - Атрибут не существует
p.voltage = 2500


Создается новый атрибут через метод `__setattr__`:

In [42]:
p.power = 1000

Метод __setattr__(power, 1000) - Изменение атрибута


In [43]:
print('p.power =', p.power)

p.power = 1000


In [44]:
p.__dict__

{'_Parrot__voltage': 2500, 'power': 1000}

## Декоратор-дескриптор property()

In [46]:
class Parrot():

    def __init__(self):
        self.__voltage = 1000

    def __get_voltage(self):
        return self.__voltage

    def __set_voltage(self, value):
        assert isinstance(value, int)
        self.__voltage = value

    # voltage здесь это открытый атрибут равный декоратору-дескриптору property().
    # В property() можно также передать метод удаления, документацию:
    voltage = property(__get_voltage, __set_voltage)


p = Parrot()

p.voltage = 1500
print('p.voltage =', p.voltage)

p.voltage = 1500


Альтернативная запись `property`:

In [47]:
class Parrot():

    voltage = property() # voltage открытый атрибут равен декоратору-дескриптору property

    def __init__(self):
        self.__voltage = 1000

    @voltage.getter
    def voltage(self):
        return self.__voltage

    @voltage.setter
    def voltage(self, value):
        assert isinstance(value, int)
        self.__voltage = value

    @voltage.deleter
    def voltage(self):
        del self.__voltage


p = Parrot()
p.voltage = 2000
print('p.voltage =', p.voltage)

p.voltage = 2000


Альтернативная запись `property`:

In [48]:
class Parrot():

    # Здесь явно атрибут voltage не декларируется, он создается декоратором @property

    def __init__(self):
        self.__voltage = 1000

    @property       # Декоратор создаст открытый атрибут voltage и геттер из этого метода:
    def voltage(self):
        return self.__voltage

    @voltage.setter # Декоратор создаст из этого метода сеттер
    def voltage(self, value):
        assert isinstance(value, int)
        self.__voltage = value

    @voltage.deleter
    def voltage(self):
        del self.__voltage


p = Parrot()
p.voltage = 3000
print('p.voltage =', p.voltage)

p.voltage = 3000


## Собственный дескриптор

Кроме встроенного дескриптора `property` можно создавать собственные:

In [56]:
# Класс дескриптор:
class VoltageValue():

    def __init__(self, value=1000):
        print('Called: __init__({})'.format(value))
        self.voltage = value

    def __get__(self, obj, obj_type):
        # obj - если равен None, то вызов атрибута идет через класс, а не через экземпляр класса
        print('Called: __get__({}, {})'.format(obj, obj_type))
        return self.voltage

    def __set__(self, obj, value):
        # obj - если равен None, то вызов атрибута идет через класс, а не через экземпляр класса
        print('Called: __set__({}, {})'.format(obj, value))
        assert isinstance(value, int)
        self.voltage = value

    def __delete__(self, obj):
        print('Called: __delete__({})'.format(obj))
        del self.voltage

In [57]:
# Обычный класс
class A():

    # Дескриптор доложен быть объявлен как атрибут класса, а не объекта!
    # Атрибут класса становится также атрибутом экземпляра этого класса!
    voltage = VoltageValue(1500)

    def __init__(self, value):
        # Не смотря на то, что voltage объявлен как атрибут класса, мы к нему обращаемся как
        # к атрибуту экземпляра класса! Такое поведение свойственно дескриптору.
        # В ином случае, вызов self.voltage создал бы атрибут экземпляра класса и стало бы 2 атрибута:
        # атрибут класса (voltage) и атрибут экземпляра класса(self.voltage) со своими значениями.
        self.voltage = value



a = A(2000)

Called: __init__(1500)
Called: __set__(<__main__.A object at 0x7f376dead748>, 2000)


Установка значения атрибута `voltage` через дескриптор `VoltageValue`:

In [58]:
a.voltage = 2500

Called: __set__(<__main__.A object at 0x7f376dead748>, 2500)


Получение значения атрибута `voltage` через дескриптор `VoltageValue`:

In [59]:
print('a.voltage =', a.voltage)

Called: __get__(<__main__.A object at 0x7f376dead748>, <class '__main__.A'>)
a.voltage = 2500


Удаление значения атрибута. Атрибут `voltage` не удаляется!

In [60]:
del a.voltage

Called: __delete__(<__main__.A object at 0x7f376dead748>)


Атрибут `voltage` класса `A` не удален после вызова через объект `del a.voltage`:

In [61]:
A.__dict__['voltage']

## Еще один пример собственного дескриптора

In [63]:
class MyNumber():

    def __init__(self, number=0):
        self.number = number

    def __get__(self, obj, obj_type):
        print('Вызван дескриптор и метод __get__({}, {})'.format(obj, obj_type))
        return self.number

    def __set__(self, obj, value):
        print('Вызван дескриптор и метод __set__({}, {})'.format(obj, value))
        self.number = value


class A():

    counter = MyNumber(10)      # MyNumber это дескриптор, который хранит значение типа int
    myvalue = MyNumber('mystr') # MyNumber это дескриптор, который хранит значение типа str



a = A()

Доступ как к атрибуту экземпляра класса:

In [65]:
print('a.counter =', a.counter)

Вызван дескриптор и метод __get__(<__main__.A object at 0x7f376de5a278>, <class '__main__.A'>)
a.counter = 10


Доступ как к атрибуту класса:

In [67]:
print('A.counter =', A.counter)

Вызван дескриптор и метод __get__(None, <class '__main__.A'>)
A.counter = 10


Какой тип будет у `A.counter` или `A.myvalue`? MyNumber? Нет:

In [70]:
type(A.counter)

Вызван дескриптор и метод __get__(None, <class '__main__.A'>)


int

In [71]:
type(A.myvalue)

Вызван дескриптор и метод __get__(None, <class '__main__.A'>)


str